In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import datetime as dt
from scipy.spatial.distance import cdist
from sklearn.cluster import DBSCAN
from sklearn.neighbors import KDTree
from sklearn.preprocessing import RobustScaler

In [3]:
plots = pd.concat((pd.read_csv(r'blue_tracks_absolute.csv'), pd.read_csv(r'red_tracks_absolute.csv')), axis=0)

In [4]:
plots = plots.drop(['Unnamed: 0', 'BlueTrackId', 'RedTrackId'], axis=1)
plots['epoch'] = (pd.to_datetime(plots['TimeStamp']) - dt.datetime(1970, 1, 1)).dt.total_seconds()

In [7]:
fig = px.scatter_3d(plots, x='Lon', y='Lat', z='Alt', template='plotly_dark', labels=None)
fig = fig.update_traces(marker={'size': 2}).update_scenes(xaxis_showticklabels=False, yaxis_showticklabels=False, zaxis_showticklabels=False)
fig.show()

In [15]:
plots = plots.sort_values('epoch').reset_index()

In [16]:
spatial_plots = RobustScaler().fit_transform(plots[['Lat', 'Lon', 'Alt', 'epoch']])

kdt = KDTree(spatial_plots, metric='euclidean')

k = 2 * spatial_plots.shape[-1] - 1

dist, ind = kdt.query(spatial_plots, k=k)

dist = np.sort(dist, axis=0)
dist = dist[:, k-1]
px.line(dist)

In [17]:
model = DBSCAN(eps=0.1581433, min_samples=spatial_plots.shape[1] * 2)
plots['clusterid'] = model.fit_predict(spatial_plots).astype('str')

In [18]:
fig = px.scatter_3d(plots, x='Lon', y='Lat', z='Alt', color='clusterid', hover_name='TimeStamp', template='plotly_dark', labels=None)
fig = fig.update_traces(marker={'size': 2}).update_scenes(xaxis_showticklabels=False, yaxis_showticklabels=False, zaxis_showticklabels=False)
fig.show()

In [11]:
distance_matrix = cdist(spatial_plots, spatial_plots)

In [150]:
def calc_angle(a, x_loc, y_loc):
    return np.arctan(np.diff(a[:, y_loc]) / np.diff(a[:, x_loc]))

In [188]:
x_loc = 0
y_loc = 1
eps = 0.1581433
alpha = 5e-33
window_size = 5

clusterid = 0
clusters = np.full(spatial_plots.shape[0], -1)

for i in range(spatial_plots.shape[0]):
    distance_from_clusters = np.zeros(clusterid + 1)
    angle_var = np.ones(clusterid + 1)

    for j in range(clusterid + 1):
        cluster_elements = np.where(clusters == j)[0]
        cluster_elements = cluster_elements if cluster_elements.shape[0] > 0 else np.zeros(1, dtype='uint')

        distance_from_clusters[j] = distance_matrix[i, cluster_elements[-1]]
        angle_var[j] = calc_angle(np.concatenate((spatial_plots[cluster_elements][-window_size:], [spatial_plots[i]])), x_loc=x_loc, y_loc=y_loc).var()
    
    min_distance = np.min(distance_from_clusters)
    min_angle_var = np.min(angle_var)

    if min_distance < eps and min_angle_var < alpha:
        clusters[i] = np.argmin(distance_from_clusters)
    else:
        clusterid += 1
        clusters[i] = clusterid

0.0
0.020897135635066868
0.020897135635066868
0.021461562688505124
0.039875361604414836
0.03018356730932145
0.06359012710445663
0.013748628296872819
0.07553345292765123
0.01925660887390627
0.08069107430409438
0.01944489663891642
0.09520583203735092
0.025122925870862935
0.11380643263838222
0.05080153299646278
0.1414380472873877
0.025652843543592208
0.16130036558814667
0.012566292609691337
0.1619521320536316
0.026063044985580763
0.18298445689329668
0.027867950459404236
0.20666478714476466
0.019533399519321804
0.2052016574425672
0.029409784350439155
0.2298569842925011
0.02836129025767489
0.23874962092675484
0.026454868740282324
0.24241084132319723
0.039093585021378746
0.26825740797880276
0.018517020486483234
0.2833429890918216
0.037686127976477364
0.3175029105561504
0.017103164350245784
0.32132215066154235
0.012212631082367972
0.3327766910440597
0.019751066416232116
0.3495329342081657
0.0169325581497642
0.3517627533900502
0.05241984589290538
0.3974816831563369
0.03136420390222107
0.393954

/tmp/ipykernel_4829/4113687480.py:2: RuntimeWarning:

invalid value encountered in divide

/tmp/ipykernel_4829/4113687480.py:2: RuntimeWarning:

divide by zero encountered in divide



In [183]:
plots['new_cluster_id'] = clusters.astype('str')

In [184]:
fig = px.scatter_3d(plots, x='Lon', y='Lat', z='Alt', color='new_cluster_id', hover_name='TimeStamp', template='plotly_dark', labels=None)
fig = fig.update_traces(marker={'size': 2}).update_scenes(xaxis_showticklabels=False, yaxis_showticklabels=False, zaxis_showticklabels=False)
fig.show()